# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

In [1]:
import string
import math
from enum import Enum

## Setting Token Enum for testing purpose

In [48]:
class Tokens(Enum):
    # Data
    Integer = int

    # Aritmetics
    Plus = "+"
    Minus = "-"
    Multiply = "*"
    Divide = "/"

    # Mehtods
    For = "for"
    LBracket = "("
    RBracket = ")"

    # Else
    EOF = None

In [21]:
list(Tokens)

[<Tokens.Integer: <class 'int'>>,
 <Tokens.Plus: '+'>,
 <Tokens.Minus: '-'>,
 <Tokens.Multiply: '*'>,
 <Tokens.Divide: '/'>,
 <Tokens.EOF: None>]

In [37]:
class Token:
    def __init__(self, type: Tokens, value) -> None:
        self.type = type
        self.value = value

## Lexer

In [108]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]

    def error():
        raise Exception('Invalid character.')
    
    # moves the pointer a character forward
    def forward(self, index) -> tuple[str,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None, index
        
        return self.input[index],index
    
    def future_token(self, index) -> tuple[Token,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None,index
        
        return self.get_token(self.input[index], index)
    
    def get_int(self, index: int) -> tuple[int,int]:
        result = ''

        # checks if there are multiple digits
        while True:
            if index < len(self.input) and self.input[index] != None and self.input[index].isdigit():
                result += self.input[index]
                _,index = self.forward(index)
            else:
                break

        return int(result),index
    
    def get_token(self, char: string, index: int) -> tuple[Token,int]:
        if char is None:
            return Token(Tokens.EOF, None),index

        if char.isdigit():
            result,index = self.get_int(index)
            return Token(Tokens.Integer, result),index

        if char == Tokens.Divide.value:
            return Token(Tokens.Divide, Tokens.Divide.value),index
        
        if char == Tokens.Multiply.value:
            return Token(Tokens.Multiply, Tokens.Multiply.value),index
        
        if char == Tokens.Plus.value:
            return Token(Tokens.Plus, Tokens.Plus.value),index
        
        if char == Tokens.Minus.value:
            return Token(Tokens.Minus, Tokens.Minus.value),index
        
        if char == ' ':
            char,index = self.forward(index)
            return self.get_token(char, index)
        
        return Token(Tokens.EOF, None),index

## Testing Lexer Types

In [110]:
lexer = lexicon("723 + 2 - 5 * 1235 / 100")

while lexer.current_char is not None:
    token,lexer.index = lexer.get_token(lexer.current_char, lexer.index)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    future_token,index = lexer.future_token(lexer.index)
    if future_token is not None:
        print("Future:" + str(future_token.type) + "\n")
    lexer.current_char, lexer.index = lexer.forward(lexer.index)

723 is of the tokentype: Tokens.Integer
Future:Tokens.Plus

+ is of the tokentype: Tokens.Plus
Future:Tokens.Integer

2 is of the tokentype: Tokens.Integer
Future:Tokens.Minus

- is of the tokentype: Tokens.Minus
Future:Tokens.Integer

5 is of the tokentype: Tokens.Integer
Future:Tokens.Multiply

* is of the tokentype: Tokens.Multiply
Future:Tokens.Integer

1235 is of the tokentype: Tokens.Integer
Future:Tokens.Divide

/ is of the tokentype: Tokens.Divide
Future:Tokens.Integer

100 is of the tokentype: Tokens.Integer


## Syntax Tree Nodes

In [111]:
class baseTree:
    pass

class numberTree(baseTree):
    pass

class binaryOperatorTree(baseTree):
    pass

## Parser

In [ ]:
class Parser:
    def __init__(self, lexer: lexicon) -> None:
        self.lexer = lexer
        self.current_token = lexer.get_token(self.lexer.current_char, self.lexer.index)
    
    def expr(self):
        node = self.term()

    def term(self):
        node = self.factor()

    def factor(self):
        pass